In [13]:
import numpy as np
from PIL import Image 
import cv2

from dataStructures import *

In [14]:
if __name__ == "__main__":
    location = "../../assets/maze.jpg"
    img = Image.open(location).convert('L')
    width, height = img.size

    img = np.array(img.point(lambda p: p > 128)) # white (p>128): 1, black(p<=128): 0
    
    # Creating Pixel objects for all white pixels
    pixMtrx = [[Pixel(r, c) if img[r][c] == 1 else None for c in range(width)] for r in range(height)]
    start = pixMtrx[227][11]
    start.dist = 0
    end = pixMtrx[211][163]

    # Checking for neighbours and connecting the graph
    for r in range(height - 1):
        for c in range(width - 1):
            if img[r, c] == 1: # if white
                if img[r, c + 1] == 1: # if right pixel is white 
                    pixMtrx[r][c].neighbours.append((r, c + 1))
                    pixMtrx[r][c + 1].neighbours.append((r, c)) # left of right pixel is white too
                if img[r + 1, c] == 1: # if bottom pixel is white 
                    pixMtrx[r][c].neighbours.append((r + 1, c))
                    pixMtrx[r + 1][c].neighbours.append((r, c)) # top of bottom pixel is white too
    for c in range(width - 1): # bottom most row
        if img[height - 1, c] == 1 and img[height - 1, c + 1] == 1: # if right pixel is white 
            pixMtrx[height - 1][c].neighbours.append((height - 1, c + 1))
            pixMtrx[height - 1][c + 1].neighbours.append((height - 1, c)) # left of right pixel is white too            
    for r in range(height - 1): # left most column
        if img[r, width - 1] == 1 and img[r + 1, width - 1] == 1: # if bottom pixel is white 
            pixMtrx[r][width - 1].neighbours.append((r + 1, width - 1))
            pixMtrx[r + 1][width - 1].neighbours.append((r, width - 1)) # top of bottom pixel is white too

    # --------------------------------DIJKSTRA'S ALGORITHM--------------------------------
    prQ = PriorityQueue(pixMtrx)
    
    while (prQ.length > 0):
        toVisit = prQ.dequeue()
        toVisit.visited = True

        for neBor in toVisit.neighbours:
            if not pixMtrx[neBor[0]][neBor[1]].visited:
                newDist = toVisit.dist + 1
                if pixMtrx[neBor[0]][neBor[1]].dist > newDist:
                    pixMtrx[neBor[0]][neBor[1]].dist = newDist
                    pixMtrx[neBor[0]][neBor[1]].parent_row = toVisit.row
                    pixMtrx[neBor[0]][neBor[1]].parent_col = toVisit.col
                index = max(index, pixMtrx[neBor[0]][neBor[1]].queueIndex)

        prQ.buildHeap(prQ.length//2 - 1) # rebuilding the heap manually        
    # ------------------------------------------------------------------------------------

    path = []
    temp = end
    while (temp != start):
        path.append((temp.row, temp.col))
        temp = pixMtrx[temp.parent_row][temp.parent_col]
    path.append((start.row, start.col))

    pic = cv2.imread(location)
    for coord in path:
        pic = cv2.circle(pic, (coord[1], coord[0]), 2, (0, 0, 255), -1)

    cv2.imshow("Solution", pic)
    cv2.waitKey(0)
